In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
from sklearn.externals import joblib
from sklearn.ensemble import RandomForestClassifier
import lightgbm as lgb
import gc

%matplotlib inline

In [2]:
plt.rc('figure',dpi=100);

In [3]:
avisos = pd.read_csv("../../Data/fiuba_entrenamiento/gian/avisos.csv")
postulaciones = pd.read_csv("../../Data/fiuba_entrenamiento/gian/postulaciones.csv")
postulantes = pd.read_csv("../../Data/fiuba_entrenamiento/gian/postulantes.csv")

postulantes['sexo']=postulantes['sexo'].astype('category')
postulantes['Doctorado']=postulantes['Doctorado'].astype('category')
postulantes['Master']=postulantes['Master'].astype('category')
postulantes['Otro']=postulantes['Otro'].astype('category')
postulantes['Posgrado']=postulantes['Posgrado'].astype('category')
postulantes['Secundario']=postulantes['Secundario'].astype('category')
postulantes['Terciario/Técnico']=postulantes['Terciario/Técnico'].astype('category')
postulantes['Universitario']=postulantes['Universitario'].astype('category')

avisos['nombre_zona']=avisos['nombre_zona'].astype('category')
avisos['tipo_de_trabajo']=avisos['tipo_de_trabajo'].astype('category')
avisos['nivel_laboral']=avisos['nivel_laboral'].astype('category')
avisos['nombre_area']=avisos['nombre_area'].astype('category')
avisos['denominacion_empresa']=avisos['denominacion_empresa'].astype('category')

postulaciones['sepostulo']=1
postulaciones.drop(['fechapostulacion','dia','mes','semana','diadelasemana','hora'],axis=1,inplace=True)

In [4]:
vistas = pd.read_csv("../../Data/fiuba_entrenamiento/gian/vistas.csv")
vistas.head()

,idAviso,timestamp,idpostulante,dia,mes,semana,diadelasemana,hora
0,1111780242,2018-02-23 18:38:13.187,YjVJQ6Z,23,2,8,Viernes,18
1,1112263876,2018-02-23 18:38:14.296,BmVpYoR,23,2,8,Viernes,18
2,1112327963,2018-02-23 18:38:14.329,wVkBzZd,23,2,8,Viernes,18
3,1112318643,2018-02-23 18:38:17.921,OqmP9pv,23,2,8,Viernes,18
4,1111903673,2018-02-23 18:38:18.973,DrpbXDP,23,2,8,Viernes,18


In [5]:
vistas.drop(['timestamp','dia','mes','semana','diadelasemana','hora'],axis=1,inplace=True)
vistas['cantidad']=0
vistas2=vistas.groupby(['idAviso','idpostulante'],as_index=False).agg({'cantidad':'count'})
vistas2=vistas2.rename(columns={'idAviso': 'idaviso'})
vistas2.head()

,idaviso,idpostulante,cantidad
0,18,BolNL,2
1,48375,RwVdKR,1
2,169730,1KjXB,2
3,169730,2AKzxa,2
4,169730,6LJ64,1


In [6]:
vistas=vistas.groupby(['idpostulante'],as_index=False).agg({'cantidad':'count'});
vistas=vistas.rename(columns={'idAviso': 'idaviso'})
vistas.head()

,idpostulante,cantidad
0,0002q,15
1,0005E,58
2,000R8,14
3,001XE,26
4,003k9,32


In [7]:
vistas2=pd.merge(vistas2,vistas,on='idpostulante',how='left')

In [8]:
vistas2.head()

,idaviso,idpostulante,cantidad_x,cantidad_y
0,18,BolNL,2,28
1,48375,RwVdKR,1,15
2,169730,1KjXB,2,99
3,169730,2AKzxa,2,2
4,169730,6LJ64,1,23


In [9]:
vistas2['cantidad']=vistas2['cantidad_x']/vistas2['cantidad_y']
vistas2.drop(['cantidad_x','cantidad_y'],axis=1,inplace=True)
vistas=vistas2
vistas2.head()

,idaviso,idpostulante,cantidad
0,18,BolNL,0.071429
1,48375,RwVdKR,0.066667
2,169730,1KjXB,0.020202
3,169730,2AKzxa,1.000000
4,169730,6LJ64,0.043478


In [10]:
postulaciones2= pd.read_csv("../../Data/fiuba_entrenamiento/gian/no-postulaciones/7.csv")
postulaciones2=postulaciones2.append(pd.read_csv("../../Data/fiuba_entrenamiento/gian/no-postulaciones/8.csv").sample(len(postulaciones2)//2).reset_index(drop=True))
print(len(postulaciones2))
postulaciones2

9906801


,idaviso,idpostulante,sepostulo
0,1112389495,5NbOjG,0
1,1112024109,1QP841b,0
2,1112490884,96XQ6N9,0
3,1111474259,qe2QJj1,0
4,1112211292,KBzbBJm,0
5,1112328485,3NzZv26,0
6,1112474766,akDjWwE,0
7,1112312385,aZpBJW,0
8,1112285628,akDWj55,0
9,1112322843,GNd0KNN,0


In [11]:
postulaciones.head()

,idaviso,idpostulante,sepostulo
0,1112257047,NM5M,1
1,1111920714,NM5M,1
2,1112346945,NM5M,1
3,1112345547,NM5M,1
4,1112237522,5awk,1


In [12]:
postulaciones=postulaciones.append(postulaciones2, ignore_index=True)
print(len(postulaciones))
postulaciones2=0
gc.collect()
postulaciones.drop_duplicates(['idaviso','idpostulante'],keep='first',inplace=True)
print(len(postulaciones))

16511335
16501975


In [13]:
postulaciones=pd.merge(postulaciones,postulantes,on='idpostulante')
postulaciones=pd.merge(postulaciones,avisos,on='idaviso')
postulaciones=pd.merge(postulaciones,vistas,on=['idaviso','idpostulante'],how='left')
postulaciones.sample(20)

,idaviso,idpostulante,sepostulo,sexo,edad,Doctorado,Master,Otro,Posgrado,Secundario,...,viajar,empresa_importante,capacitacion,remuneracion_pretendida,graduados,estudiantes,conocimientos_especificos,buen_ambiente,discriminacion,cantidad
11915837,1112497102,ow5BdaZ,0,MASC,32.0,-,-,Graduado,Graduado,Graduado,...,False,True,False,True,True,True,False,False,False,NaN
7978504,1111358945,aaYlzJ,1,FEM,30.0,-,-,Graduado,-,Graduado,...,False,False,False,False,False,False,True,False,True,NaN
11875954,1112336377,YNpNKr,0,MASC,40.0,-,-,-,-,-,...,False,False,False,False,False,False,False,False,False,NaN
2449467,1112298966,2zrpOk1,1,FEM,21.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,False,NaN
15408290,1112481324,jkj6bqB,0,FEM,28.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,True,False,NaN
12745206,1112369484,5bXXKZ,1,FEM,32.0,-,-,-,-,-,...,False,False,False,False,False,False,False,False,False,NaN
866895,1112355134,mzbGGrG,1,FEM,27.0,-,-,-,-,-,...,False,False,True,False,False,False,False,True,False,0.021053
2208102,1112322693,Bm5NK4a,1,FEM,21.0,-,-,Graduado,-,Graduado,...,False,True,True,False,False,False,False,True,False,NaN
11396631,1112293428,VNPjE1j,1,FEM,22.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,False,NaN
5801598,1112322551,96MeGz1,0,MASC,32.0,-,-,-,-,-,...,False,True,False,False,False,False,False,False,False,NaN


In [14]:
postulaciones.cantidad=postulaciones.cantidad.fillna(0)
postulaciones.sample(20)

,idaviso,idpostulante,sepostulo,sexo,edad,Doctorado,Master,Otro,Posgrado,Secundario,...,viajar,empresa_importante,capacitacion,remuneracion_pretendida,graduados,estudiantes,conocimientos_especificos,buen_ambiente,discriminacion,cantidad
11218862,1112354284,QNPXE5N,1,FEM,34.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,False,0.000000
1205569,1112342289,5mkxmZv,0,FEM,24.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,False,0.000000
9165430,1111393842,dYj30vD,1,MASC,23.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,False,0.076923
7939891,1112285328,wVk4aom,0,FEM,21.0,-,-,-,-,-,...,False,False,True,False,False,False,False,True,False,0.000000
5927950,1112368302,8MrjG2O,0,FEM,33.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,False,0.000000
1928704,1112425374,3Nz3JZx,1,FEM,26.0,-,-,Graduado,-,Graduado,...,False,True,False,False,False,False,False,False,False,0.008929
8578090,1112386928,5NKLVq,0,MASC,39.0,-,-,-,Graduado,Graduado,...,False,False,False,False,False,False,False,False,False,0.000000
10424434,1112418662,QNle9Jz,0,MASC,22.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,False,0.000000
5743232,1112336668,RzrrWeo,1,FEM,18.0,-,-,-,-,En Curso,...,False,False,False,False,False,False,False,False,False,0.000000
8839032,1112271813,6rLeGwR,1,MASC,22.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,False,0.000000


In [15]:
train, test=train_test_split(postulaciones,test_size=0.05)
print("Train: ",len(train),"Test: ",len(test))
features=list(postulaciones.columns)
features.remove('idaviso')
features.remove('idpostulante')
features.remove('sepostulo')
features.remove('titulo')

x_train=train[features]
y_train=train['sepostulo']

x_test=test[features]
y_test=test['sepostulo']
postulaciones=0
vistas2=0

Train:  15281844 Test:  804308


In [18]:
gc.collect()

0

In [19]:
c=lgb.LGBMClassifier(learning_rate=0.01,objective='binary',num_leaves=6000,max_depth=13,n_estimators=1200,colsample_bytree=0.8,n_jobs=-1,random_state=0,silent=False,subsample=0.8,sumsample_freq=0.5)

dt=c.fit(x_train,y_train,eval_set=[(x_test,y_test)],verbose=True,early_stopping_rounds=1)
#c = joblib.load("../../Data/fiuba_entrenamiento/gian/modelos/....pkl")

[1]	valid_0's binary_logloss: 0.689678
Training until validation scores don't improve for 1 rounds.
[2]	valid_0's binary_logloss: 0.685191
[3]	valid_0's binary_logloss: 0.683426
[4]	valid_0's binary_logloss: 0.679414
[5]	valid_0's binary_logloss: 0.675876
[6]	valid_0's binary_logloss: 0.674183
[7]	valid_0's binary_logloss: 0.67045
[8]	valid_0's binary_logloss: 0.666251
[9]	valid_0's binary_logloss: 0.663529
[10]	valid_0's binary_logloss: 0.659489
[11]	valid_0's binary_logloss: 0.656518
[12]	valid_0's binary_logloss: 0.652898
[13]	valid_0's binary_logloss: 0.649972
[14]	valid_0's binary_logloss: 0.647791
[15]	valid_0's binary_logloss: 0.643981
[16]	valid_0's binary_logloss: 0.64056
[17]	valid_0's binary_logloss: 0.637229
[18]	valid_0's binary_logloss: 0.633734
[19]	valid_0's binary_logloss: 0.630207
[20]	valid_0's binary_logloss: 0.627898
[21]	valid_0's binary_logloss: 0.625361
[22]	valid_0's binary_logloss: 0.62365
[23]	valid_0's binary_logloss: 0.620889
[24]	valid_0's binary_logloss: 

In [20]:
score=c.score(x_test,y_test)*100
print(score)

83.72464279852991


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [21]:
joblib.dump(c, "../../Data/fiuba_entrenamiento/gian/modelos/LGBMV17.pkl")

['../../Data/fiuba_entrenamiento/gian/modelos/LGBMV17.pkl']

In [22]:
prediccion = pd.read_csv("../../Data/fiuba_entrenamiento/test_final_100k.csv")
print(len(prediccion))
prediccion=pd.merge(prediccion,postulantes,on='idpostulante',how='inner')
print(len(prediccion))
prediccion=pd.merge(prediccion,avisos,on='idaviso',how='inner')
print(len(prediccion))
prediccion=pd.merge(prediccion,vistas,on=['idaviso','idpostulante'],how='left')
prediccion.cantidad=prediccion.cantidad.fillna(0)
prediccion.sample(10)

100000
100000
100000


,id,idaviso,idpostulante,sexo,edad,Doctorado,Master,Otro,Posgrado,Secundario,...,viajar,empresa_importante,capacitacion,remuneracion_pretendida,graduados,estudiantes,conocimientos_especificos,buen_ambiente,discriminacion,cantidad
76355,36374,1112411686,kPXdNwJ,FEM,23.0,-,-,-,-,-,...,False,False,False,False,False,False,False,False,False,0.100000
1758,86771,1112466769,bOjJ06E,NO_DECLARA,23.0,-,-,-,-,Graduado,...,False,True,False,False,False,True,False,False,False,0.007576
33602,8129,1111753681,ow2bAG0,MASC,23.0,-,-,-,-,Graduado,...,False,False,False,True,False,False,False,False,False,0.200000
23951,71666,1112462863,8wYPeW,NO_DECLARA,200.0,-,-,-,-,-,...,False,True,False,False,False,False,False,False,False,0.000000
25705,49083,1112437340,59LrwG,MASC,29.0,-,-,-,-,-,...,False,True,False,False,False,False,False,False,False,0.000000
27593,17834,1112205158,bOjBxAb,MASC,25.0,-,-,-,-,En Curso,...,False,False,False,False,False,False,False,False,False,0.000000
68949,62271,1112456031,1loXZj,FEM,54.0,-,-,Graduado,-,Graduado,...,True,False,False,False,False,False,False,False,False,0.000000
23920,71451,1112462348,6r5bwQx,FEM,26.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,False,0.010000
94125,93363,1112479408,6klvZv,MASC,45.0,-,Graduado,-,En Curso,Graduado,...,False,False,True,False,False,True,False,False,False,0.000000
15751,67647,1112460365,pzdRz3r,FEM,40.0,-,-,-,-,-,...,False,True,False,False,False,False,False,False,False,0.007634


In [23]:
x_final=prediccion[features]
y_final=c.predict_proba(x_final)
y_final

array([[0.88718673, 0.11281327],
       [0.86570813, 0.13429187],
       [0.87033981, 0.12966019],
       ...,
       [0.93948007, 0.06051993],
       [0.9294001 , 0.0705999 ],
       [0.97738199, 0.02261801]])

In [24]:
y_final2=[]
print(c.classes_)
print(len(y_final))
for elem in y_final:
    y_final2.append(elem[1])
print(len(y_final2))

[0 1]
100000
100000


In [25]:
prediccion['sepostulo']=y_final2

In [26]:
prediccion.head()

,id,idaviso,idpostulante,sexo,edad,Doctorado,Master,Otro,Posgrado,Secundario,...,empresa_importante,capacitacion,remuneracion_pretendida,graduados,estudiantes,conocimientos_especificos,buen_ambiente,discriminacion,cantidad,sepostulo
0,0,739260,6M9ZQR,FEM,42.0,Graduado,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,0.0,0.112813
1,1,739260,6v1xdL,MASC,30.0,-,-,Graduado,-,-,...,False,False,False,False,False,False,False,False,0.0,0.134292
2,2,739260,ezRKm9,FEM,36.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,0.0,0.129660
3,3,758580,1Q35ej,MASC,68.0,-,-,-,Graduado,Graduado,...,False,False,False,False,False,False,False,False,0.0,0.006733
4,4,758580,EAN4J6,FEM,32.0,-,-,-,-,-,...,False,False,False,False,False,False,False,False,0.0,0.005104


In [31]:
#prediccion.drop(features,axis=1,inplace=True)
prediccion.drop(['idaviso','idpostulante','titulo'],axis=1,inplace=True)
prediccion.head()

,id,sepostulo
0,0,0.112813
1,1,0.134292
2,2,0.129660
3,3,0.006733
4,4,0.005104


In [32]:
prediccion.to_csv("../../Data/fiuba_entrenamiento/gian/predicciones/prediccion35.csv",index=False)

In [33]:
prediccion.sepostulo.mean()

0.5160103078244644